In [24]:
import boto3
import datetime
import pytz

In [25]:
s3 = boto3.resource('s3')
s3client = boto3.client('s3')

In [26]:
target_bucket = 'aii-data'


In [27]:
target_date = datetime.datetime(2019,5,31,0,0,0,0, pytz.UTC)
old_file_set = []

In [28]:
def set_archive_tag(bucket, key):
    response = s3.meta.client.put_object_tagging(
        Bucket=bucket,
        Key=key,
        Tagging={
            'TagSet': [
                {
                    'Key': 'storage_level',
                    'Value': 'archive'
                    },
                ]
            },
        )

In [23]:
count = 0
limit = 10000000000

paginator = s3client.get_paginator('list_objects')
pageresponse = paginator.paginate(Bucket=target_bucket)
#out_f.write('+++++\n')
#out_f.write(str(bucket) + '\n')
#out_f.write('+++++\n')

# PageResponse Holds 1000 objects at a time and
# will continue to repeat in chunks of 1000.
for pageobject in pageresponse:
    #out_f.write('----\n')
    #out_f.write('----\n')
    if 'Contents' in pageobject:
        for item in pageobject["Contents"]:
            # Skip small files
            if item['Size'] < 1024 * 1024:
                continue
                
            
        
            if item['LastModified'] <= target_date:
                #print("found")
                #print(item)
                set_archive_tag(target_bucket, item['Key'])
            else: 
                continue    
            count += 1
            if count > limit:
                break
        if count > limit:
            break
    if count > limit:
        break

KeyboardInterrupt: 

In [11]:
print(item)

{'Key': 'v1/experimental/terrestrial/L0/UK_Test/20200701/g5ra_log_1593626014.txt', 'LastModified': datetime.datetime(2020, 7, 1, 18, 12, 4, tzinfo=tzutc()), 'ETag': '"ddbbbacd919ebe4ac3b488f7021ec090"', 'Size': 887, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'scanthespectrum', 'ID': 'cce8acfe91eb3a7432b8a2f8b610b830625a027c6c47dd3436ae147a71b71d16'}}


In [35]:
count = 0
limit = 1000000000
tagged_count = 0
not_tagged_count = 0
total_count = 0
tagged_list = []

paginator = s3client.get_paginator('list_objects')
pageresponse = paginator.paginate(Bucket=target_bucket)
#out_f.write('+++++\n')
#out_f.write(str(bucket) + '\n')
#out_f.write('+++++\n')

# PageResponse Holds 1000 objects at a time and
# will continue to repeat in chunks of 1000.
tagged_size = 0
for pageobject in pageresponse:
    #out_f.write('----\n')
    #out_f.write('----\n')
    if 'Contents' in pageobject:
        for item in pageobject["Contents"]:
            not_tagged_count += 1
            total_count += 1
            # Skip small files
            if item['Size'] < 1024 * 1024:
                continue

            if item['LastModified'] <= target_date:
                tags = s3.meta.client.get_object_tagging(
                    Bucket=target_bucket,
                    Key=item['Key']
                    )
                #print("found")
                if 'TagSet' in tags:
                    if tags['TagSet']:
                        if tags['TagSet'][0]['Value'] != 'archive':
                            print("Missed File")
                        else:
                            tagged_size += item['Size']
                            tagged_count += 1
                        tagged_list.append(
                            tuple([
                                target_bucket,
                                item['Key'],
                                item['Size'],
                                item['StorageClass'],
                                item['LastModified']]
                            ))
                            
                not_tagged_count -= 1
                #print(item)
                #print(tags)
      


            

                

            count += 1
            if count > limit:
                break
        if count > limit:
            break
    if count > limit:
        break

In [36]:
tagged_list[:10]



[('aii-data',
  'aii-data/aii-data-inventory/data/0108c3d4-ec3b-4564-aa19-874cfd1418f2.csv.gz',
  1896127,
  'GLACIER',
  datetime.datetime(2019, 4, 15, 1, 32, 23, tzinfo=tzutc())),
 ('aii-data',
  'aii-data/aii-data-inventory/data/08368155-b130-4212-a6b2-8a1462083c11.csv.gz',
  1806713,
  'GLACIER',
  datetime.datetime(2019, 4, 1, 9, 15, 42, tzinfo=tzutc())),
 ('aii-data',
  'aii-data/aii-data-inventory/data/0d099c26-76c0-44dc-9034-2b9e39dbc8b9.csv.gz',
  1647733,
  'GLACIER',
  datetime.datetime(2019, 4, 29, 7, 32, 23, tzinfo=tzutc())),
 ('aii-data',
  'aii-data/aii-data-inventory/data/10042854-394e-437d-a6b4-57c229ac849d.csv.gz',
  1757763,
  'GLACIER',
  datetime.datetime(2019, 5, 5, 22, 33, 56, tzinfo=tzutc())),
 ('aii-data',
  'aii-data/aii-data-inventory/data/16c33ac3-08c6-41de-8567-272976e6e9ba.csv.gz',
  1741048,
  'GLACIER',
  datetime.datetime(2018, 11, 26, 8, 49, 41, tzinfo=tzutc())),
 ('aii-data',
  'aii-data/aii-data-inventory/data/1975796d-a543-44c3-897f-dd9888c85e2b.csv

In [37]:
len(tagged_list)

155052

In [38]:
tags['TagSet']

[{'Key': 'storage_level', 'Value': 'archive'}]

In [39]:
tags['TagSet'][0]['Value']
                      

'archive'

In [40]:
tagged_size

140287069395035